<a href="https://colab.research.google.com/github/ExtrApostroPhe/Summer_Project/blob/main/TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import ssl
import re
import string
import os

from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction import text 
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


from sklearn.cluster import KMeans 
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords 

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv1D, Dense, MaxPool1D, Flatten, Input, Dropout
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stw = set(stopwords.words('english'))

drive.mount('/content/drive', force_remount=True)  
os.chdir('/content/drive/My Drive/news_ipm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Mounted at /content/drive


In [5]:
# Read Dataset
data_env = pd.read_csv('./train_data/environment.txt', header = None, sep = "\n")  # 这里只获取部分数据
data_health = pd.read_csv('./train_data/health.txt', header = None, sep = "\n")
data_hk = pd.read_csv('./train_data/hk_macao.txt', header = None, sep = "\n")
data_inno = pd.read_csv('./train_data/innovation.txt', header = None, sep = "\n")
data_military = pd.read_csv('./train_data/military.txt', header = None, sep = "\n")
data_notional = pd.read_csv('./train_data/notional_affairs.txt', header = None, sep = "\n")
data_society = pd.read_csv('./train_data/society.txt', header = None, sep = "\n")

data_env[1] = "0"
data_health[1] = "1"
data_hk[1] = "2"
data_inno[1] = "3"
data_military[1] = "4"
data_notional[1] = "5"
data_society[1] = "6"

# print(type(data))
# print(data_env, data_health)
# print(data_health)
data = pd.concat([data_env, data_health, data_hk, data_inno, data_military, data_notional, data_society ], axis = 0, ignore_index = True)
print(data)

                                                       0  1
0          'Chief' system boosts water quality of rivers  0
1                    County tackles soil erosion head-on  0
2         30% cut expected in new high-emission projects  0
3             High-altitude wind farm in Tibet starts up  0
4               System proves Fuzhou's green credentials  0
...                                                  ... ..
23861    Ministry: 'Genocide in Xinjiang' another US lie  6
23862             Yunnan city reports new COVID-19 cases  6
23863  Judges urge trademark applicants to be more ho...  6
23864                  Cherry trees of a different color  6
23865  New book on Party history released at Beijing ...  6

[23866 rows x 2 columns]


In [6]:
X = data[0]
Y = data[1]

In [7]:
tfidf = TfidfVectorizer(stop_words=stw,max_df=0.5)
X = tfidf.fit_transform(X).toarray()


x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [22]:
def insert_text():
    X_temp = []
    for x in X:
        X_temp.append(x)
    return X_temp

def filter_text(text):
    str = re.sub('[^a-zA-Z]', ' ', text)
    return re.sub(r'\s+', ' ', str)

def lower_text(text):
    return text.lower()

def lemmatization_text(text_cut_list):
    wnl = WordNetLemmatizer()
    return [wnl.lemmatize(n) for n in text_cut_list]

def text_cut(text):
    return re.findall('[a-zA-z]+', text)

def stopwords_text(text_cut_list):
    Stop_Word_list = stopwords.words("english")
    return [n for n in text_cut_list if n not in Stop_Word_list]

article = insert_text()
X = []
# print(article)
for content in article:
    content = filter_text(content)
    content = lower_text(content)
#     print(content)
    word_cut = lemmatization_text(text_cut(content))
    word_cut = stopwords_text(word_cut)
#     print(word_cut)
    content = ''
    for w in word_cut:
        content = content + w + ' '
    X.append(content)


In [26]:
nb_model = MultinomialNB(alpha=0.001)
nb_model.fit(x_train,y_train)
predict_test = nb_model.predict(x_test)
print("多项式朴素贝叶斯文本分类的准确率为：",metrics.accuracy_score(predict_test,y_test))
print(metrics.classification_report(predict_test, y_test))
print(metrics.confusion_matrix(predict_test, y_test))

多项式朴素贝叶斯文本分类的准确率为： 0.6606145251396648
              precision    recall  f1-score   support

           0       0.65      0.67      0.66       845
           1       0.69      0.71      0.70      1192
           2       0.75      0.74      0.75      1175
           3       0.76      0.73      0.74      1250
           4       0.56      0.69      0.62       240
           5       0.64      0.59      0.61      1312
           6       0.50      0.51      0.51      1146

    accuracy                           0.66      7160
   macro avg       0.65      0.66      0.66      7160
weighted avg       0.66      0.66      0.66      7160

[[569  18  13  51   4  74 116]
 [ 15 852  71  66   7  64 117]
 [ 19  64 875  27  16 106  68]
 [ 53  89  36 908  17  49  98]
 [  4   5   9  22 165  22  13]
 [ 85  74  86  45  65 771 186]
 [126 128  80  83  19 120 590]]


In [25]:
#bernoulli朴素贝叶斯
ber_model = BernoulliNB(alpha=0.001)
ber_model.fit(x_train,y_train)
ber_predict = ber_model.predict(x_test)
print("bernoulli贝叶斯文本分类的准确率为：",metrics.accuracy_score(ber_predict,y_test))
print(metrics.classification_report(ber_predict, y_test))
print(metrics.confusion_matrix(ber_predict, y_test))

bernoulli贝叶斯文本分类的准确率为： 0.6804469273743017
              precision    recall  f1-score   support

           0       0.68      0.67      0.68       879
           1       0.72      0.73      0.72      1220
           2       0.78      0.80      0.79      1139
           3       0.75      0.74      0.75      1228
           4       0.63      0.66      0.65       280
           5       0.65      0.61      0.63      1279
           6       0.51      0.53      0.52      1135

    accuracy                           0.68      7160
   macro avg       0.68      0.68      0.68      7160
weighted avg       0.68      0.68      0.68      7160

[[592  17  11  55   4  79 121]
 [ 11 886  64  69   7  63 120]
 [ 10  43 915  20  13  84  54]
 [ 53  86  33 906  15  46  89]
 [  4   6  14  24 185  30  17]
 [ 80  73  68  43  52 782 181]
 [121 119  65  85  17 122 606]]


In [29]:
gauss_model = GaussianNB()
gauss_model.fit(x_train,y_train)
gauss_predict = ber_model.predict(x_test)
print("GaussianNB贝叶斯文本分类的准确率为：",metrics.accuracy_score(gauss_predict,y_test))
print(metrics.classification_report(gauss_predict, y_test))
print(metrics.confusion_matrix(gauss_predict, y_test))

GaussianNB贝叶斯文本分类的准确率为： 0.6804469273743017
              precision    recall  f1-score   support

           0       0.68      0.67      0.68       879
           1       0.72      0.73      0.72      1220
           2       0.78      0.80      0.79      1139
           3       0.75      0.74      0.75      1228
           4       0.63      0.66      0.65       280
           5       0.65      0.61      0.63      1279
           6       0.51      0.53      0.52      1135

    accuracy                           0.68      7160
   macro avg       0.68      0.68      0.68      7160
weighted avg       0.68      0.68      0.68      7160

[[592  17  11  55   4  79 121]
 [ 11 886  64  69   7  63 120]
 [ 10  43 915  20  13  84  54]
 [ 53  86  33 906  15  46  89]
 [  4   6  14  24 185  30  17]
 [ 80  73  68  43  52 782 181]
 [121 119  65  85  17 122 606]]


In [30]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [31]:
#构建CNN分类模型(LeNet-5)

model = Sequential()

model.add(Dense(512, input_shape = (x_train.shape[1], ), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(7, activation = 'softmax'))

early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=2)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data = (x_test, y_test))

Epoch 1/10
131/131 [==============================] - 16s 117ms/step - loss: 1.2128 - accuracy: 0.5766 - val_loss: 0.7428 - val_accuracy: 0.7575
Epoch 2/10
131/131 [==============================] - 15s 115ms/step - loss: 0.5006 - accuracy: 0.8368 - val_loss: 0.7348 - val_accuracy: 0.7566
Epoch 3/10
131/131 [==============================] - 14s 109ms/step - loss: 0.2963 - accuracy: 0.9009 - val_loss: 0.8366 - val_accuracy: 0.7332
Epoch 4/10
131/131 [==============================] - 14s 110ms/step - loss: 0.1909 - accuracy: 0.9361 - val_loss: 0.9500 - val_accuracy: 0.7244
Epoch 5/10
131/131 [==============================] - 14s 109ms/step - loss: 0.1365 - accuracy: 0.9557 - val_loss: 1.0545 - val_accuracy: 0.7173
Epoch 6/10
131/131 [==============================] - 14s 109ms/step - loss: 0.1073 - accuracy: 0.9638 - val_loss: 1.1538 - val_accuracy: 0.7089
Epoch 7/10
131/131 [==============================] - 14s 109ms/step - loss: 0.0998 - accuracy: 0.9662 - val_loss: 1.1820 - val_ac